In [ ]:
# Instalamos las librerías necesarias
!pip install requests
!pip install pandas

In [ ]:
# Importamos  librerías
import requests
import pandas as pd
import json
from datetime import datetime

In [ ]:
# Configuración de token y la versión de la API
ACCESS_TOKEN = "IGAATJSKBr5P9BZAE5NblBIMzNqeVJLdXd6c3did3dSYllPa0F3T0p5TlYxWDY0Wk9wbXI4UkRiSzhPOUl2U1VGRzNlZAk44S3pLa2k5UlNLVjhQTU5OVTUxZA2hCUDZAXdTBEeHNodkI2eUVKLXM0LTgwRUdPdnlTZAW9CcC1VWFlmZAwZDZD"
API_VERSION = "v22.0"

# Verificación del token
if not ACCESS_TOKEN or ACCESS_TOKEN == "PASTE_NEW_TOKEN_HERE":
    print("¡Error! Configura un token válido en ACCESS_TOKEN.")
else:
    print("Token configurado:", ACCESS_TOKEN[:10], "... (ocultado por seguridad)")
    print("Versión de la API configurada:", API_VERSION)

Token configurado: IGAATJSKBr ... (ocultado por seguridad)
Versión de la API configurada: v22.0


In [ ]:
# Función para hacer solicitudes a la API: se define una función reusable para enviar solicitudes a la API.
def fetch_instagram_data(endpoint, params, version=API_VERSION):
    url = f"https://graph.instagram.com/{version}/{endpoint}"
    params['access_token'] = ACCESS_TOKEN
    response = requests.get(url, params=params)
    print(f"URL solicitada: {response.url}")  # Para depuración
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

print("Función fetch_instagram_data definida")

Función fetch_instagram_data definida


In [ ]:
# Probamos el token con datos básicos del perfil: Se solicita datos básicos del perfil (ID, username, seguidores, número de publicaciones) para verificar que el token funciona.
profile_params = {
    "fields": "id,username,followers_count,media_count"
}
profile_data = fetch_instagram_data("me", profile_params)

if profile_data:
    print("Datos del perfil:")
    print(json.dumps(profile_data, indent=4))
else:
    print("Sin información.")

URL solicitada: https://graph.instagram.com/v22.0/me?fields=id%2Cusername%2Cfollowers_count%2Cmedia_count&access_token=IGAATJSKBr5P9BZAE5NblBIMzNqeVJLdXd6c3did3dSYllPa0F3T0p5TlYxWDY0Wk9wbXI4UkRiSzhPOUl2U1VGRzNlZAk44S3pLa2k5UlNLVjhQTU5OVTUxZA2hCUDZAXdTBEeHNodkI2eUVKLXM0LTgwRUdPdnlTZAW9CcC1VWFlmZAwZDZD
Datos del perfil:
{
    "id": "9567080036669811",
    "username": "lenceria_santaevacali",
    "followers_count": 3596,
    "media_count": 72
}


In [ ]:
# Función para manejar la paginación y obtener todas las publicaciones
def fetch_all_instagram_media(params, version=API_VERSION):
    all_media = []
    next_url = f"https://graph.instagram.com/{version}/me/media"

    while next_url:
        response = requests.get(next_url, params=params)
        print(f"URL solicitada: {response.url}")
        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        if "data" not in data:
            print("Sin información")
            break

        all_media.extend(data["data"])
        # Verifica si hay más páginas
        next_url = data.get("paging", {}).get("next", None)
        params = {}  # Para las siguientes solicitudes, los parámetros están en la URL de paginación

    return all_media

# Configuramos los campos que queremos extraer
media_params = {
    "fields": "id,media_type,media_url,timestamp,like_count,comments_count,children{media_type}",
    "access_token": ACCESS_TOKEN
}

# Extraemos todas las publicaciones
media_data = fetch_all_instagram_media(media_params)

if media_data:
    print("Número total de publicaciones extraídas:", len(media_data))
else:
    print("Sin información")

#Función fetch_all_instagram_media: Maneja la paginación automáticamente. La API devuelve un enlace a la "siguiente página" (paging.next) si hay más publicaciones. La función sigue ese enlace hasta que no haya más datos.

URL solicitada: https://graph.instagram.com/v22.0/me/media?fields=id%2Cmedia_type%2Cmedia_url%2Ctimestamp%2Clike_count%2Ccomments_count%2Cchildren%7Bmedia_type%7D&access_token=IGAATJSKBr5P9BZAE5NblBIMzNqeVJLdXd6c3did3dSYllPa0F3T0p5TlYxWDY0Wk9wbXI4UkRiSzhPOUl2U1VGRzNlZAk44S3pLa2k5UlNLVjhQTU5OVTUxZA2hCUDZAXdTBEeHNodkI2eUVKLXM0LTgwRUdPdnlTZAW9CcC1VWFlmZAwZDZD
URL solicitada: https://graph.instagram.com/v22.0/17841439137400116/media?fields=id%2Cmedia_type%2Cmedia_url%2Ctimestamp%2Clike_count%2Ccomments_count%2Cchildren%7Bmedia_type%7D&access_token=IGAATJSKBr5P9BZAE5NblBIMzNqeVJLdXd6c3did3dSYllPa0F3T0p5TlYxWDY0Wk9wbXI4UkRiSzhPOUl2U1VGRzNlZAk44S3pLa2k5UlNLVjhQTU5OVTUxZA2hCUDZAXdTBEeHNodkI2eUVKLXM0LTgwRUdPdnlTZAW9CcC1VWFlmZAwZDZD&limit=25&after=QVFIUkRqOGxxV19Nc2p3LWVBbzlwQlllR0ZAmMHk0N1lUVlFsbEloYW04NXFSc2JLZAnd5U25IV3RYZAGtVazJlM2xWY0ZAWdkh2ZAHJUdUdnYVVjbzlicUQtQ0ZA3
URL solicitada: https://graph.instagram.com/v22.0/17841439137400116/media?fields=id%2Cmedia_type%2Cmedia_url%2Ctimestamp%2Cli

In [ ]:
# Convertimos publicaciones a DataFrame y guardamos en CSV
if media_data:
    df_media = pd.DataFrame(media_data)
    df_media["timestamp"] = pd.to_datetime(df_media["timestamp"])
    df_media.to_csv("instagram_media.csv", index=False)
    print("Publicaciones guardadas en 'instagram_media.csv'")
else:
    print("No se guardaron datos.")

#Ajustamos para usar media_data directamente (ya que fetch_all_instagram_media devuelve una lista).
#Guardamos los datos en CSV

Publicaciones guardadas en 'instagram_media.csv'


In [ ]:
if media_data:

# Aseguramos que 'like_count' y 'comments_count' sean numéricos
    df_media['like_count'] = pd.to_numeric(df_media['like_count'], errors='coerce')
    df_media['comments_count'] = pd.to_numeric(df_media['comments_count'], errors='coerce')

# Creamos una columna para interactividad total (likes + comentarios)
    df_media['interactivity'] = df_media['like_count'] + df_media['comments_count']

# Número total de likes y comentarios
    total_likes = df_media['like_count'].sum()
    total_comments = df_media['comments_count'].sum()
    print(f"\nTotal de likes en todas las publicaciones: {total_likes}")
    print(f"Total de comentarios en todas las publicaciones: {total_comments}")

# Promedio de likes y comentarios por publicación
    avg_likes = df_media['like_count'].mean()
    avg_comments = df_media['comments_count'].mean()
    print(f"Promedio de likes por publicación: {avg_likes:.2f}")
    print(f"Promedio de comentarios por publicación: {avg_comments:.2f}")

# Publicación con mayor interactividad
    most_interactive_post = df_media.loc[df_media['interactivity'].idxmax()]
    print("\nPublicación con mayor interactividad:")
    print(f"ID: {most_interactive_post['id']}")
    print(f"Tipo de medio: {most_interactive_post['media_type']}")
    print(f"URL: {most_interactive_post['media_url']}")
    print(f"Fecha: {most_interactive_post['timestamp']}")
    print(f"Likes: {most_interactive_post['like_count']}")
    print(f"Comentarios: {most_interactive_post['comments_count']}")
    print(f"Interactividad total: {most_interactive_post['interactivity']}")

# Publicación con mínima interactividad
    least_interactive_post = df_media.loc[df_media['interactivity'].idxmin()]
    print("\nPublicación con mínima interactividad:")
    print(f"ID: {least_interactive_post['id']}")
    print(f"Tipo de medio: {least_interactive_post['media_type']}")
    print(f"URL: {least_interactive_post['media_url']}")
    print(f"Fecha: {least_interactive_post['timestamp']}")
    print(f"Likes: {least_interactive_post['like_count']}")
    print(f"Comentarios: {least_interactive_post['comments_count']}")
    print(f"Interactividad total: {least_interactive_post['interactivity']}")

# Conteo y análisis por tipo de publicación (videos, imágenes, carruseles)
# Conteo de cada tipo
    media_type_counts = df_media['media_type'].value_counts()
    print("\nConteo de publicaciones por tipo:")
    print(f"Videos: {media_type_counts.get('VIDEO', 0)}")
    print(f"Imágenes: {media_type_counts.get('IMAGE', 0)}")
    print(f"Carruseles: {media_type_counts.get('CAROUSEL_ALBUM', 0)}")

# Análisis por tipo de medio (likes totales, promedio, conteo)
    media_type_analysis = df_media.groupby('media_type').agg({
    'like_count': ['sum', 'mean'],  # Total y promedio de likes
    'comments_count': ['sum', 'mean'],  # Total y promedio de comentarios
    'interactivity': 'mean',  # Promedio de interactividad
    'id': 'count'  # Número de publicaciones

}).reset_index()

# Renombramos las columnas para claridad
media_type_analysis.columns = ['media_type', 'total_likes', 'avg_likes',
                               'total_comments', 'avg_comments', 'avg_interactivity', 'count']

print("\nAnálisis detallado por tipo de publicación:")
print(media_type_analysis)


Total de likes en todas las publicaciones: 2175
Total de comentarios en todas las publicaciones: 73
Promedio de likes por publicación: 30.21
Promedio de comentarios por publicación: 1.01

Publicación con mayor interactividad:
ID: 18071542846726129
Tipo de medio: VIDEO
URL: https://scontent-iad3-1.cdninstagram.com/o1/v/t2/f2/m86/AQOTEtcPrCgdLL3CYtgF4rGM_7YqZgB4F-42e0YZaGKSTUIIKOcQjupqmGdMGZ5I2-3C0Y2IkZPVm5pp8zY6CZRTZX5MOueO_cX4qu8.mp4?_nc_cat=110&_nc_sid=5e9851&_nc_ht=scontent-iad3-1.cdninstagram.com&_nc_ohc=1I4vyNyqzekQ7kNvgE9tGXd&efg=eyJ2ZW5jb2RlX3RhZyI6Inhwdl9wcm9ncmVzc2l2ZS5JTlNUQUdSQU0uQ0xJUFMuQzMuNzIwLmRhc2hfYmFzZWxpbmVfMV92MSIsInhwdl9hc3NldF9pZCI6MTI2NzAzMzc2NDQ4OTQzMiwidmlfdXNlY2FzZV9pZCI6MTAwOTksImR1cmF0aW9uX3MiOjc1LCJ1cmxnZW5fc291cmNlIjoid3d3In0%3D&ccb=17-1&vs=ee7987a3ceea9a1a&_nc_vs=HBksFQIYUmlnX3hwdl9yZWVsc19wZXJtYW5lbnRfc3JfcHJvZC9DNDQ1NDQ1MDNENTFDOTg1NzZFN0EzNjk2QzM3OTBBOF92aWRlb19kYXNoaW5pdC5tcDQVAALIAQAVAhg6cGFzc3Rocm91Z2hfZXZlcnN0b3JlL0dIZDU3QnRtbFVTOEFsY0hBRlhhYVNFOTd

# **CONEXIÓN GOOGLE SHEETS**

In [ ]:
# Instalamos las librerías necesarias para Google Sheets
!pip install gspread
!pip install google-auth

# Importamos las librerías necesarias para Google Sheets
import gspread
from google.colab import auth
from google.auth import default

# Autenticamos al usuario
auth.authenticate_user()

# Obtenemos las credenciales
creds, _ = default()

# Conectamos con gspread usando las credenciales
gc = gspread.authorize(creds)

# Abrimos la hoja de cálculo por su URL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1rI9x6x00aAR4E6EGaq48it660tk1YDxHmwOWb-zU8N4/edit?gid=0#gid=0"
spreadsheet = gc.open_by_url(spreadsheet_url)

# Seleccionamos la hoja específica (por ejemplo, la primera hoja, "Sheet1")
worksheet = spreadsheet.worksheet("DATOS EXTRAIDOS API")  # Seleccionamos la hoja por su nombre

In [ ]:
# Aseguramos que df_media existe
if 'df_media' in globals():
    # Convertimos timestamp a string para que sea compatible con Google Sheets
    df_media["timestamp"] = df_media["timestamp"].astype(str)

    # Si la columna 'children' existe, la convertimos a string (puede ser una lista)
    if "children" in df_media.columns:
        df_media["children"] = df_media["children"].astype(str)

    # Convertimos el DataFrame a una lista de listas para escribir en Google Sheets
    data_to_write = [df_media.columns.tolist()]  # Encabezados
    data_to_write.extend(df_media.values.tolist())  # Filas de datos

    # Limpiamos la hoja existente (esto para cargar datos actualizados)
    worksheet.clear()

    # Escribimos los datos en la hoja
    worksheet.update("A1", data_to_write)

    print("Publicaciones guardadas en Google Sheets")
else:
    print("Error: No se encontró el DataFrame 'df_media'. Asegúrate de que las celdas anteriores se hayan ejecutado correctamente.")

<ipython-input-17-19f957a38c03>:18: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("A1", data_to_write)


Publicaciones guardadas en Google Sheets
